In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Wrangle

In [9]:
#print('* Turn off the scrolling feature of Jupyter Notebook')
# Import
print('* Import libraries')
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Markdown
from IPython.display import display, HTML
from pprint import pprint

import lib.p3_basic_summary as summary
from p3_CellCounts import CellCounts
#
from p3_Transforms import Transforms, get_raw_transforms_json
from p3_SummaryQuery import SummaryQuery
from p3_Labels import Labels, get_feature_labels
import lib.p3_table as mdtb
#
import lib.p3_helper_functions as helper
import p3_pie as piie
#
import p3_bars as bars

print('* Add magic keyword: ','%matplotlib inline' )
%matplotlib inline
print('* load utility classes')


#--------------------------------- utility classs
labels = Labels(get_feature_labels()) # load verbose feature labels
cc = CellCounts()

#--------------------------------- Open Clean data
print('* Open clean data')
transforms = Transforms(get_raw_transforms_json('conf.raw.transforms.json'))
const_appt = 0
const_patient = 1
const_neighbourhood =2
convert_appt_types = None
convert_patient_types = { # configure type changes
    'conversions': [
        {'column':'patient_id','to':'int'}, 
        {'column':'appointments','to':'int'}, 
        {'column':'no_shows','to':'int'}, 
        {'column':'appointment_group','to':'int'},  
        {'column':'scholarship','to':'int'},
        {'column':'hipertension','to':'int'},
        {'column':'diabetes','to':'int'},
        {'column':'alcoholism','to':'int'},
        {'column':'gender','to':'int'},
    ]
}
convert_neighbourhood_types = None
sources = [
    {'transform':'appt_transform','type_conversion': convert_appt_types},
    {'transform':'patient2_transform','type_conversion': convert_patient_types},
    {'transform':'neighbourhood1_transform','type_conversion': convert_neighbourhood_types},
]

big_data = helper.open_source_data(sources,transforms)

df_appt = big_data[const_appt]
df_patient = big_data[const_patient]
df_neighbourhood = big_data[const_neighbourhood]


#-------------------------- Summerize Neighbourhood Data
print('* summerize Neighbourhood data...')
neighbourhood_summary, neighbourhood_no_show_summary,neighbourhood_show_up_summary \
    = helper.get_neighbourhood_summaries(df_neighbourhood)

#--------------------------------- Summerize Appointment Data
print('* summerize appointment data...')
appt_summary, appt_no_show_summary,appt_show_up_summary \
    = helper.get_appt_summaries(df_appt)

#-------------------------- Summerize Patient Data
print('* summerize patient data...')
patient_summary, patient_no_show_summary,patient_show_up_summary \
    = helper.get_patient_summaries(df_patient)

print('* Done')

* Import libraries


ModuleNotFoundError: No module named 'p3_basic_summary'

# Data Inspection

In [ ]:
# df_appointments.describe

## Observations
* 
*
*

# Questions
#### _1. Does the number of visits matter with regards to skipping appointments?_
#### _2. Do all patients have maladies?_
#### _3. What is the most common reason for a visit?_
#### _4. Which neighbourhood has the highest number of  patients?_
#### _5. What is the most common patient malady by neighbourhood?_
#### _6. What is the most common day of the week for appointments?_
#### _7. What is most common time of day for appointments?_
#### _8. Describe the relationship between patient appointments and no-shows_
#### _9. Describe the relationship between patient age and no-shows_
#### _10. Describe the relationship between patient appointments and age for shows and no-shows?_


## _Does the number of visits matter with regards to skipping appointments?_

Here I'm looking for people who are making appointments for preventitive care, such as annual checkups.  I expect preventative patients are a different population from those that have a diagnosis.

> **Strategy** Count the number of visits per patient and examine the spread. 

>**Answer**
* **Yes, the number of visits matters.**  First time patients are vulnerable to skipping. About 40% of all no-shows are first time visitors.
* 71.7% of patients have never skipped an appointment, whereas 28.3% have skipped at least once.
* Of all of the appointments, 11.4% have skipped thier first appointment. This can also be stated as 40% of all skippers have skipped on thier very first visit.
   
 

In [ ]:
helper.graph_Patient_Attendance_and_Skipped_Visits(df_patient)

## _Do all patients have maladies?_

Here I'm looking for people who are making appointments for preventitive care, such as annual checkups.  I expect preventative patients are a different population from those that have a diagnosis.

> **Strategy** Query Patient data for any patient where all maladies are zero.
* Maladie are hipertension, diabetes, alcoholism, and handcap

>**Answer**
* **No, not all patients have a malady**. 
    * 47788 patients have no recorded malady. 
    * 76.764% of patients have no health data on which to base a no-show prediction.



In [ ]:
helper.qraph_Do_all_patient_have_maladies(df_patient)

### Observations


## _What is the most common Malady of Visiting Patients?_
Patients can be grouped by malady.  These malady groups may be a source for predicting no-shows.

> **Strategy** Group Patients to find most common maladys.
* Maladies are hipertension, diabetes, alcoholism, and handcap

>**Answer**
* **Hipertension is the most common malady **. 
    * 76.764% (47788) of Patients have no malady 
    * 19.605% (12205) of Patients have hipertension 
    * 7.082% (4409) of Patients have diabetes 
    * 2.419% (1506) of Patients have alcoholism 
    * 1.815% (1130) of Patients have a handcap 

In [ ]:
helper.graph_What_is_the_most_common_Malady_of_Visiting_Patients(df_patient)

## _Which neighbourhood has the highest number of  patients?_


> **Strategy** Group Patients by neighbourhood.
* 
neighbourhood
                 
>**Answer**
* **JARDIM CAMBURI has the most patients ** 
    * 4192 patients in JARDIM CAMBURI
    * No visible pattern in the geographic distribution of appointments 
    

In [ ]:
helper.neighbourhood_percents(df_patient)

In [ ]:
helper.map_neighbourhood_appointments(df_neighbourhood)

## _What is the most common patient malady by neighbourhood?_


> **Strategy**: Use graphs for each neighbourhood to search out the malady with the highest occurance.
*  Use dataframe.groupby and dataframe.idxmax(axis=1)
                 
>**Answer**:
* **Hipertension is the most common malady had by a patient health care visit ** 
    

### Graphs of maladies by neighbourhoods 
* The answer is here but not obvious...let's try a map (see below)

In [ ]:
helper.graph_Most_Common_Neighbourhood_Malady(df_patient)

### The answer becomes clear with a map
* **Hipertension** is the most common malady across all neighbourhoods!

In [ ]:
helper.map_Most_Common_Neighbourhood_Malady(df_neighbourhood,df_patient)

# What is the most common day of the week for appointments?

> **Strategy** 
* Breakdown the scheduled day of the appointment into year, month, day of week.
* use graph to find the most common time of day

> **Answer**:
* **Tuesday (1)** has the highest number of scheduled visits
* Saturday has too few visits to register on the graph
* Sunday's have no patient appointments.
* Looks like there might be a pattern emerging in the begining of the week (Mon, Tue, Wed). More appointments are scheduled at the begining verses the end of the week (Thr,Fri)


In [ ]:
helper.graph_Scheduled_Day_of_Week(df_appt)

# _What is most common time of day for appointments?_

> **Strategy**: Show graph, list numbers and use get_favorites function to pick one or more peaks.

>**Answer**: 
* **7:00** is the favored hour for appointments.
* Looks like a pattern is emerging for scheduled_hour. High number of appointments in the early morning, decreases by noon and rises after noon and finally decreases toward the end of the day. The pattern echos for show and no-shows.

In [ ]:
helper.graph_What_is_Most_Common_Time_of_Day_for_Appointments(df_appt)


# _Describe the relationship between patient appointments and no-shows_


> **Strategy**: if a big patter exists, it should show itself by relationship. Make scatter plots, fit line to scatter, interperate the line slope. 

>**Answer**: 
* The relationship between Patient "appointments" in not visually revealing.  
* equation:  y = 0.006974429953645027x + 0.18459023215908543
* description:  As "appointments" increases "no_show_ratio" increases "slowly"

In [ ]:
helper.graph_Describe_the_relationship_between_patient_appointments_and_no_shows(df_patient) 


# _Describe the relationship between patient age and no-shows_


> **Strategy**: if a big pattern exists, it should show itself by relationship. Make scatter plots, fit line to scatter, interperate the line slope. 

>**Answer**: 
* The relationship between Patient "age" and "no-shows" in not visually revealing.  
* equation:  y = -0.0011679911130957098x + 0.24109070967559182
* description:  As "age" increases "no_show_ratio" decreases "slowly"



### First what do No-Show vs Appointments look like?

In [ ]:
helper.graph_Describe_the_relationship_between_patient_age_and_no_shows(df_patient) 

### Second??????????????????????

# _Describe the relationship between patient appointments and age for shows and no-shows?_


> **Strategy**: if a big patter exists, it should show itself by relationship. Make scatter plots, fit line to scatter, interperate the line slope. 

>**Answer**: 
* Age vs Appointments isn't giving up any visible insight.
    * equation:  y = -0.0026724480188155455x + 1.6829472643933194
    * description:  As "age" increases "appointments" decreases "slowly"
* Looking at the patients that show up for their appointments yields almost no relationship
    * equation:  y = -0.0026724480188155455x + 1.6829472643933194
    * description:  As "age" increases "appointments" decreases "slowly"
* Looking at the patients that are no-show, again yield almost no relationship
    * equation:  y = -0.00181172532204667x + 2.6296590402358966
    * description:  As "age" increases "appointments" decreases "slowly"

### First what does Age vs Appointments look like?

In [ ]:
helper.graph_First_what_does_Age_vs_Appointments_look_like(df_patient)  

### What do shows look like?

In [ ]:
helper.graph_What_do_shows_look_like(df_patient)

### What do no-shows look like?

In [ ]:
helper.graph_What_do_no_shows_look_like(df_patient)    